In [93]:
import pandas as pd
import os

In [94]:
allgsdf = pd.read_csv("../../resources/20180622processedGoldStandardTopics.tsv.gz", delimiter="\t")
gsdf = allgsdf.drop(["title", "abstract", "major_mesh", "minor_mesh"], axis=1)

# Read all results

In [106]:
def calculateRelFoundCounts(dir):
    # Read all results in the given result directory
    resultfiles = sorted(os.listdir(dir))
    # Read the result files as DataFrames into a map
    resultdfmap = []
    for f in resultfiles:
        resultdfmap.append(pd.read_csv(dir+f, delimiter="\t", names=["topic", "Q0", "docid", "rank", "score", "run"]))
    # Create a DataFrame multiindexed with the file name (because those are the keys of the DF maps)
    resultmultidxdf = pd.concat(resultdfmap)
    resultmultidxdf.set_index(["run"], inplace=True)
    
    # Create a duplication of the relevant GS document to match the results
    gsreldocs = gsdf.query("relevance_score > 0")[["trec_topic_number", "trec_doc_id"]]
    l = []
    for experiment in set(resultmultidxdf.index):
        idx = pd.Index([experiment]*len(gsreldocs), name="run")
        gscopy = gsreldocs.copy()
        gscopy.index = idx
        l.append(gscopy)
    gsdfs = pd.concat(l)
    
    # Merge the duplicated GS with the results
    # With a `left` join, thus eliminating all irrelevant documents.
    relmerge = pd.merge(gsdfs, resultmultidxdf, how="left", left_on=["run", "trec_topic_number", "trec_doc_id"], right_on=["run", "topic", "docid"])
    relmerge.set_index("trec_topic_number", append="True", inplace=True)
    
    # Count the number of found documents per run and topic    
    countsruntopic = relmerge.groupby(["run", "trec_topic_number"]).count()
    countsruntopic = countsruntopic.drop(["Q0", "docid", "rank", "score"],axis=1)
    countsruntopic.columns = ["relgs", "relfound"]
    
    # return the left-merged data and the counts
    return relmerge,countsruntopic

In [96]:
def prepareStats(path):
    """
    Reads a single stats CSV file, excludes the 'all' row and converts the topic numbers to ints.
    Then sets the Topic columns as the new index.
    Returns a DataFrame indexed by the non-'all' topics.
    """
    df = pd.read_csv(path).query("Topic != 'all'")
    df["Topic"] = df["Topic"].astype(int)
    df.sort_values(by="Topic", inplace=True)
    df = df.set_index("Topic")
    return df

In [123]:
measures = ["infNDCG"]

def getMeanStatsPerRun(statspath):
    """
    Reads a directory of stat CSV files. Concatenates all the DataFrames and calculates the means for all
    score measurements of the topics per run, effectively returning the 'all' row for each run. Note,
    however, that really just the mean over the measures is given which should be the 'all' value but the actual
    'all' value is not used here.
    Returns only those measures defined in the 'measures' list at the beginning of this cell.
    """
    statfiles = sorted(list(filter(lambda f: f.endswith(".csv") and not "gspm" in f, os.listdir(statspath))))
    runstatsmap = {}
    for stat in statfiles:
        df = prepareStats(statspath+stat)
        run = stat.replace("OFFICIAL_", "").replace(".csv", "")
        runstatsmap[run] = df
    allstats = pd.concat(runstatsmap)
    allstats.index.names = ["run", "Topic"]
    allstats = allstats[measures]
    meanstats = allstats.mean(level="run")
    return meanstats

In [169]:
def getRelFoundWithMeanRank(resultspath, statspath):
    gsleftmerged,counts = calculateRelFoundCounts(resultspath)
    meanstats = getMeanStatsPerRun(statspath)
    counts   = counts.sum(level="run").sort_values("relfound")
    merge     = pd.merge(counts, meanstats, on="run")
    meanranks = gsleftmerged["rank"].dropna().mean(level="run")
    stdranks  = gsleftmerged["rank"].dropna().std(level="run")
    merge["meanrank"] = meanranks
    merge["stdrank"] = stdranks
    return merge.sort_values("relfound")

In [170]:
getRelFoundWithMeanRank("../../results_nowr/", "../../stats_nowr/")

,relgs,relfound,infNDCG,meanrank,stdrank
run,,,,,
baselinewr,3875,2898,0.444370,587.314010,1000.123354
genesonly,3875,2989,0.326227,949.805621,1517.841900
baseline,3875,3228,0.393510,760.732962,1216.394281
diseases,3875,3324,0.344807,923.909446,1408.774797
genes,3875,3472,0.371300,935.288594,1419.388215
genesnodesc,3875,3481,0.383080,877.321172,1308.963602
genesplus,3875,3482,0.377230,922.338024,1378.319279
genedis,3875,3541,0.315567,1153.769557,1656.313819
